<a href="https://colab.research.google.com/github/3DLables/3DLabeler/blob/master/MouseSkullKPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mouse Skull Model



In [0]:
import numpy as np
from tqdm import tqdm
import os

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
skull_1179 = np.load('/gdrive/My Drive/Colab Notebooks/MouseSkulls/' +
                     'ReducedData/skull_1187.npy',
                     allow_pickle=True)

In [4]:
skull_1179[0].shape

(128, 128, 128)

In [5]:
skull_1179[1].shape

(5, 3)

In [0]:
basepath = '/gdrive/My Drive/Colab Notebooks/MouseSkulls/ReducedData/'
file_list = os.listdir(basepath)

In [19]:
skulls = []
points = []
for file in tqdm(file_list):
    file = np.load(basepath+file, allow_pickle=True)
    skulls.append(file[0])
    points.append(file[1])

100%|██████████| 218/218 [02:41<00:00,  1.31it/s]


In [0]:
test = np.array(skulls)

In [23]:
test.shape

(218, 128, 128, 128)